In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import zipfile
import urllib.request
import os
import matplotlib.pyplot as plt
import shutil
from PIL import Image, ImageDraw
import random
import re
from scipy import spatial

# General Helper functions

In [ ]:
def normalize_posenet_vector(vecteur):
    
    x_components, y_components = zip(*vecteur)
    
    x_components, y_components = np.array(x_components), np.array(y_components)
    
    
    max_amplitude = max(max(x_components)-min(x_components),max(y_components)-min(y_components))
    
    amplification_factor = 1/max_amplitude
    
    x_components = amplification_factor*x_components
    y_components = amplification_factor*y_components
    
    def mean_extrema(array):
        return np.mean((max(array),min(array)))
    
    x_components, y_components =\
    x_components-mean_extrema(x_components) + 1/2, y_components-mean_extrema(y_components) + 1/2
    
    return list(zip(x_components, y_components))

In [ ]:
def create_json_df(directory_path, json_name, normalise_function):
    
    rows = []
    
    with open(json_name, "r") as json:
        
        text = json.read()
        
        rows = []
        
        for item in eval(text):
            image_name = item.get("filename")
            
            image_id = int(re.match("\d+",image_name).group())
            
            poses = item.get("poses")
            
            normalized_poses = list(map(normalise_function, poses))
            
            if os.path.exists(directory_path+"/"+image_name):           
                rows.append({"image_identifer":image_id,\
                             "image_name": image_name, "poses":poses, "norm_poses":normalized_poses})
    
    return pd.DataFrame(rows)

In [ ]:
def show_pictures(poses, name, directory_path, verbose = True):
    
    path = directory_path+"/"+name
    
    im = Image.open(path)
    
    if verbose:
        c = 0
        colors = ["red","yellow","green","blue","grey"]
        for pose in poses:

            color = colors[c]

            width, height = im.size

            draw = ImageDraw.Draw(im)

            pose = list(map(lambda tup : (width*tup[0], height*tup[1]), pose))

            for point in pose[:5]:
                x = point[0]
                y = point[1]
                r = 5
                draw.ellipse([x-r,y-r,x+r,y+r],color,color)

            for i in (5,6,11,12):
                draw.line([(pose[i][0],pose[i][1]),(pose[i+2][0],pose[i+2][1]),(pose[i+4][0],pose[i+4][1])],
                          fill = color, width = 5)

            c += 1
    
    plt.figure(figsize = (10,10))
    plt.imshow(np.asarray(im))
    plt.show()

In [ ]:
df = create_local_df("images_sample", "rijk.json", normalize_posenet_vector)

In [ ]:
#df.apply(lambda row: show_pictures(row["poses"],row["image_name"], "images_sample"), axis = 1)

# Angular similarity

In [ ]:
def divergence(pose1, pose2, fcompare, flist):
    
    acc = 0
    
    for f in flist:
        acc += fcompare(f(pose1),f(pose2))
        
    return acc

def min_divergence(pose1, pose_list, fcompare, flist):
    
    current_min = np.Infinity
    
    for pose in pose_list:
        current_min = min(current_min, divergence(pose1,pose,fcompare,flist))
        
    return current_min

In [ ]:
def fcompare_absolute_divergence(v1,v2):
    return abs(v1-v2)

In [ ]:
def angle_to_horizontal(tup1,tup2):
    """Return the angle to horizontal for a vector bounded by points tup1 and tup2"""
    vector = np.array(tup2)-np.array(tup1)
    
    if vector[0] != 0:
        return np.arctan(vector[1]/vector[0])
    else:
        return np.arctan(vector[1]*np.Infinity)
    

In [ ]:
LEFT_HUMERUS = (5,7)
RIGHT_HUMERUS = (6,8)
LEFT_FOREARM = (7,9)
RIGHT_FOREARM = (8,10)

LEFT_THIGH = (11,13)
RIGHT_THIGH = (12,14)
LEFT_SHIN = (13,15)
RIGHT_SHIN = (14,16)

SHOULDERS = (5,6)
LEFT_SIDE = (5,11)
RIGHT_SIDE = (6,12)
PELVIS = (11,12)

In [ ]:
def f_angle_segment(pose, body_part):
    """Return the angle of a body segment"""
    return angle_to_horizontal(pose[body_part[0]],pose[body_part[1]])

In [ ]:
def f_angle_chunk(pose,body_part_1,body_part_2):
    """Return the angle of a large body part bounded by 2 segments"""
    mean_vector = ((np.array(pose[body_part_1[0]]) + np.array(pose[body_part_2[0]]))/2,
                   (np.array(pose[body_part_1[1]]) + np.array(pose[body_part_2[1]]))/2)
    
    return angle_to_horizontal(mean_vector[0],mean_vector[1])

In [ ]:
def fcompare_cosine_similarity(pose1, pose2):
    return spatial.distance.cosine(np.array(pose1).flatten(),np.array(pose2).flatten())

In [ ]:
df.head()

In [ ]:
def give_n_best_match(input_name, input_df, target_df, fcompare, flist, n=3):
    """
    Give the n best match for input image
    
    input_df and target_df must have an "image_name" and "poses" field
    """
    input_pose = input_df.loc[df["image_name"] == input_name]["norm_poses"].values[0][0]
    
    divergences = target_df["norm_poses"].apply(lambda poses: min_divergence(input_pose, poses, fcompare, flist))
    
    top_indexes = divergences.sort_values(ascending = True).index[:n]
    
    return target_df.iloc[top_indexes]

In [ ]:
df = df[df.poses.apply(lambda poses: len(poses) > 0)].reset_index()


def show_n_best_matchs(input_name,input_df,target_df,fcompare,flist,n=3):

    input_row = input_df[input_df["image_name"] == input_name]

    results = give_n_best_match(input_name,input_df,target_df,fcompare,flist,n)

    print("Input")
    input_row.apply(lambda row: show_pictures(row["poses"],row["image_name"], "images_sample"), axis = 1)
    print("Results")
    results.apply(lambda row: show_pictures(row["poses"],row["image_name"], "images_sample"), axis = 1)

In [ ]:
show_n_best_matchs("5037.jpg",df,df,fcompare_cosine_similarity,[lambda x:x],n=5)

In [ ]:
arms_list = [lambda pose : f_angle_segment(pose, LEFT_HUMERUS), lambda pose : f_angle_segment(pose, RIGHT_HUMERUS),
            lambda pose : f_angle_segment(pose, LEFT_FOREARM),lambda pose : f_angle_segment(pose, RIGHT_FOREARM)]
show_n_best_matchs("5037.jpg",df,df,fcompare_absolute_divergence,arms_list,n=5)

In [ ]:
# A faire:
# Introduire une mesure tableau de face ou de profil (ratio longueur du torse Vs largeur)
# Introduire des fonctions pour la position de la tête
# Introduire des poids pour les différentes fonctions de la liste
# Réintroduire une notion de direction dans la mesure de l'angle
# (sinon un bras plié dans un sens ou dans l'autre à 180 degré sera compté la même chose !)